<a href="https://colab.research.google.com/github/DAUN3046/AI-Agent-Exercise/blob/main/03.%20Custom_Tool%EB%A1%9C_%EC%A3%BC%EC%8B%9D%EB%B6%84%EC%84%9D_%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8_%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **API 키 및 기타 설정**

In [1]:
# CrewAI 설치
!pip install crewai transformers accelerate --quiet
!pip install crewai litellm duckduckgo-search
!pip install together
!pip install 'crewai[tools]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires chromadb<0.6.0,>=0.5.10, but you have chromadb 1.0.4 which is incompatible.
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached chromadb-0.5.23-py3-none-any.whl (628 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: chromadb
    Found existing installation: chromadb 1.0.4
    Uninstalling chromadb-1.0.4:
      Successfully uninstalle

In [7]:
!pip install langchain_anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 6.4 MB/s eta 0:00:00


In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["SERPER_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ""
os.environ['ANTHROPIC_API_KEY'] = ""

### **필수 라이브러리 불러오기**

In [12]:
from crewai import Agent, Task, Crew, Process, LLM
# from crewai_tools import tool, SerperDevTool
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool, tool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

### **툴 설정**

In [14]:
#웹 검색 툴 설정
search_tool = SerperDevTool()

#재무 분석 툴 설정
@tool("Updated Comprehensive Stock Analysis")
def comprehensive_stock_analysis(ticker: str) -> str:
    """
    주어진 주식 티커에 대한 업데이트된 종합적인 재무 분석을 수행합니다.
    최신 주가 정보, 재무 지표, 성장률, 밸류에이션 및 주요 비율을 제공합니다.
    가장 최근 영업일 기준의 데이터를 사용합니다.

    :param ticker: 분석할 주식의 티커 심볼
    :return: 재무 분석 결과를 포함한 문자열
    """
    def format_number(number):
        if number is None or pd.isna(number):
            return "N/A"
        return f"{number:,.0f}"

    def calculate_growth_rate(current, previous):
        if previous and current and previous != 0:
            return (current - previous) / abs(previous) * 100
        return None

    def format_financial_summary(financials):
        summary = {}
        for date, data in financials.items():
            date_str = date.strftime('%Y-%m-%d')
            summary[date_str] = {
                "총수익": format_number(data.get('TotalRevenue')),
                "영업이익": format_number(data.get('OperatingIncome')),
                "순이익": format_number(data.get('NetIncome')),
                "EBITDA": format_number(data.get('EBITDA')),
                "EPS(희석)": f"${data.get('DilutedEPS'):.2f}" if pd.notna(data.get('DilutedEPS')) else "N/A"
            }
        return summary

    ticker = yf.Ticker(ticker)
    historical_prices = ticker.history(period='1d', interval='1d')
    latest_price = historical_prices['Close'].iloc[-1]
    latest_time = historical_prices.index[-1].strftime('%Y-%m-%d %H:%M:%S')

    # 연간 및 분기별 재무제표 데이터 가져오기
    annual_financials = ticker.get_financials()
    quarterly_financials = ticker.get_financials(freq="quarterly")
    balance_sheet = ticker.get_balance_sheet()

    # 주요 재무 지표 (연간)
    revenue = annual_financials.loc['TotalRevenue', annual_financials.columns[0]]
    cost_of_revenue = annual_financials.loc['CostOfRevenue', annual_financials.columns[0]]
    gross_profit = annual_financials.loc['GrossProfit', annual_financials.columns[0]]
    operating_income = annual_financials.loc['OperatingIncome', annual_financials.columns[0]]
    net_income = annual_financials.loc['NetIncome', annual_financials.columns[0]]
    ebitda = annual_financials.loc['EBITDA', annual_financials.columns[0]]

    # 부채비율 계산
    total_assets = balance_sheet.loc['TotalAssets', balance_sheet.columns[0]]
    total_liabilities = balance_sheet.loc['TotalLiabilitiesNetMinorityInterest', balance_sheet.columns[0]]
    debt_ratio = (total_liabilities / total_assets) * 100 if total_assets != 0 else None

    # 주요 비율 계산
    gross_margin = (gross_profit / revenue) * 100 if revenue != 0 else None
    operating_margin = (operating_income / revenue) * 100 if revenue != 0 else None
    net_margin = (net_income / revenue) * 100 if revenue != 0 else None

    # 성장성 지표 계산 (연간)
    revenue_growth = calculate_growth_rate(revenue, annual_financials.loc['TotalRevenue', annual_financials.columns[1]])
    net_income_growth = calculate_growth_rate(net_income, annual_financials.loc['NetIncome', annual_financials.columns[1]])

    # 주당 지표
    diluted_eps = annual_financials.loc['DilutedEPS', annual_financials.columns[0]]

    # 분기별 데이터 분석
    quarterly_revenue = quarterly_financials.loc['TotalRevenue', quarterly_financials.columns[0]]
    quarterly_net_income = quarterly_financials.loc['NetIncome', quarterly_financials.columns[0]]

    quarterly_revenue_growth = calculate_growth_rate(
        quarterly_revenue,
        quarterly_financials.loc['TotalRevenue', quarterly_financials.columns[1]]
    )
    quarterly_net_income_growth = calculate_growth_rate(
        quarterly_net_income,
        quarterly_financials.loc['NetIncome', quarterly_financials.columns[1]]
    )

    return {
        "현재 주가":{
            "현재 주가": latest_price,
            "기준 시간": latest_time
        },
        "연간 데이터": {
            "매출": format_number(revenue),
            "매출원가": format_number(cost_of_revenue),
            "매출총이익": format_number(gross_profit),
            "영업이익": format_number(operating_income),
            "순이익": format_number(net_income),
            "EBITDA": format_number(ebitda),
            "매출총이익률": f"{gross_margin:.2f}%" if gross_margin is not None else "N/A",
            "영업이익률": f"{operating_margin:.2f}%" if operating_margin is not None else "N/A",
            "순이익률": f"{net_margin:.2f}%" if net_margin is not None else "N/A",
            "매출 성장률": f"{revenue_growth:.2f}%" if revenue_growth is not None else "N/A",
            "순이익 성장률": f"{net_income_growth:.2f}%" if net_income_growth is not None else "N/A",
            "희석주당순이익(EPS)": f"${diluted_eps:.2f}" if diluted_eps is not None else "N/A",
            "부채비율": f"{debt_ratio:.2f}%" if debt_ratio is not None else "N/A",
        },
        "분기 데이터": {
            "매출": format_number(quarterly_revenue),
            "순이익": format_number(quarterly_net_income),
            "매출 성장률(QoQ)": f"{quarterly_revenue_growth:.2f}%" if quarterly_revenue_growth is not None else "N/A",
            "순이익 성장률(QoQ)": f"{quarterly_net_income_growth:.2f}%" if quarterly_net_income_growth is not None else "N/A",
        },
        "연간 재무제표 요약": format_financial_summary(annual_financials),
        "분기별 재무제표 요약": format_financial_summary(quarterly_financials),
    }

### **에이전트 설정**

In [15]:
# 현재 시간 가져오기
current_time = datetime.now()

llm = LLM(model = "openai/gpt-4o-mini")
invest_llm = LLM(model="anthropic/claude-3-5-sonnet-20240620")
# 재무 분석가
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# 시장 분석가
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# 위험 분석가
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# 투자 전문가
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [19]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
    return ticker

def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다.
        주요 재무 지표에 집중하세요.
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서.
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며,
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )

    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다.
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요.
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서.
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고,
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다.
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요.
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서.
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )

    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다.
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보
           - 회사명, 티커, 현재 주가, 목표주가, 투자의견 등

        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리

        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등

        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망

        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석

        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션

        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거

        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열

        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """
    )

    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

In [20]:
ticker = get_user_input()
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

투자 자문을 구하고 싶은 기업명을 입력해주세요: AAPL


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1eb6895f-cf63-4635-9587-363beccd2925                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 4fbd6dfb-ec0f-4ffe-add3-ce4206c78ca7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 4fbd6dfb-ec0f-4ffe-add3-ce4206c78ca7
       Status: Executing Task...
    └── 🤖 Agent: Financial Analyst
            Status: In Progress

# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다.
        주요 재무 지표에 집중하세요.
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2025년 04월 11일


...

# Agent: Investment Advisor
## Task: AAPL의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다.
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요.날짜: 2025년 04월 11일


# Agent: Investment Advisor
## Final Answer: 
# Apple Inc. (AAPL) 투자 분석 보고서

## 1. 제목 및 기본 정보
- 회사명: Apple Inc.
- 티커: AAPL
- 현재 주가: 190.42 USD (2025년 4월 10일 기준)
- 목표주가: 220 USD
- 투자의견: 매수

## 2. 요약(Executive Summary)
Apple Inc.는 지속적인 혁신과 강력한 브랜드 가치를 바탕으로 견고한 재무 성과를 보이고 있습니다. 높은 수익성과 성장 잠재력을 고려할 때, 장기 투자자들에게 매력적인 투자 대상입니다. 다만, 순이익 감소와 높은 부채 비율 등의 위험 요인에 주의가 필요합니다.

## 3. 기업 개요
Apple Inc.는 세계적인 기술 기업으로, 아이폰, 맥북, 아이패드 등의 하드웨어와 iOS, macOS 등의 소프트웨어, 그리고 다양한 서비스를 제공합니다. 1976년 설립 이후, 혁신적인 제품으로 기술 산업을 선도하고 있으며, 글로벌 스마트폰 시장에서 높은 점유율을 유지하고 있습니다.

## 4. 산업 및 시장 분석
- 서비스 부문의 성장: Apple의 서비스 부문 매출이 연간 12-15% 성장할 것으로 예상됩니다.
- 구독 모델 확대: Apple Music, Apple TV+, iCloud 등의 구독 서비스 확대로 고객 유지율 향상이 기대됩니다.
- 신규 제품 라인: AR 안경, 전기차 기술 등 새로운 제품군 개발로 추가 성장 동력 확보가 예상됩니다.
- 신흥 시장 진출: 선진국 시장 포화에 따라 신흥국 시장에서의 성장 잠재력이 높습니다.

## 5. 재무 분석
- 매출: 391,035,

🚀 Crew: crew
├── 📋 Task: 4fbd6dfb-ec0f-4ffe-add3-ce4206c78ca7
│      Assigned to: Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Analyst
│           Status: ✅ Completed
├── 📋 Task: dd2adac5-472b-4a70-83f2-f0fc2ad0ac2e
│      Assigned to: Market Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Analyst
│           Status: ✅ Completed
├── 📋 Task: 25281022-91eb-413f-b652-f3d93d574618
│      Assigned to: Risk Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Analyst
│           Status: ✅ Completed
└── 📋 Task: 9344cf77-14c9-4e06-a06e-e84b0a2b2a23
       Status: Executing Task...
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 4fbd6dfb-ec0f-4ffe-add3-ce4206c78ca7
│      Assigned to: Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Analyst
│           Status: ✅ Completed
├── 📋 Task: dd2adac5-472b-4a70-83f2-f0fc2ad0ac2e
│      Assigned to: Market Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Market Analyst
│           Status: ✅ Completed
├── 📋 Task: 25281022-91eb-413f-b652-f3d93d574618
│      Assigned to: Risk Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Analyst
│           Status: ✅ Completed
└── 📋 Task: 9344cf77-14c9-4e06-a06e-e84b0a2b2a23
       Assigned to: Investment Advisor
       Status: ✅ Completed
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9344cf77-14c9-4e06-a06e-e84b0a2b2a23                                                                     │
│  Agent: Investment Advisor                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1eb6895f-cf63-4635-9587-363beccd2925                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# Apple Inc. (AAPL) 투자 분석 보고서

## 1. 제목 및 기본 정보
- 회사명: Apple Inc.
- 티커: AAPL
- 현재 주가: 190.42 USD (2025년 4월 10일 기준)
- 목표주가: 220 USD
- 투자의견: 매수

## 2. 요약(Executive Summary)
Apple Inc.는 지속적인 혁신과 강력한 브랜드 가치를 바탕으로 견고한 재무 성과를 보이고 있습니다. 높은 수익성과 성장 잠재력을 고려할 때, 장기 투자자들에게 매력적인 투자 대상입니다. 다만, 순이익 감소와 높은 부채 비율 등의 위험 요인에 주의가 필요합니다.

## 3. 기업 개요
Apple Inc.는 세계적인 기술 기업으로, 아이폰, 맥북, 아이패드 등의 하드웨어와 iOS, macOS 등의 소프트웨어, 그리고 다양한 서비스를 제공합니다. 1976년 설립 이후, 혁신적인 제품으로 기술 산업을 선도하고 있으며, 글로벌 스마트폰 시장에서 높은 점유율을 유지하고 있습니다.

## 4. 산업 및 시장 분석
- 서비스 부문의 성장: Apple의 서비스 부문 매출이 연간 12-15% 성장할 것으로 예상됩니다.
- 구독 모델 확대: Apple Music, Apple TV+, iCloud 등의 구독 서비스 확대로 고객 유지율 향상이 기대됩니다.
- 신규 제품 라인: AR 안경, 전기차 기술 등 새로운 제품군 개발로 추가 성장 동력 확보가 예상됩니다.
- 신흥 시장 진출: 선진국 시장 포화에 따라 신흥국 시장에서의 성장 잠재력이 높습니다.

## 5. 재무 분석
- 매출: 391,035,000,000 USD (전년 대비 2.02% 증가)
- 영업이익: 123,216,000,000 USD (영업이익률 31.51%)
- 순이익: 93,736,000,000 USD (순이익률 23.97%)
- EBITDA: 134,661,000,000 USD
- EPS(희석): 6.08 USD

수익성 지표는 전반적으로 우수하나, 순이익 성장률이 -3.36%로 감소세를 보이고 있어 주의가 필요합니다.

## 6. 밸류에이션
- P/E: 31.32 (업계 평균 대비 약간 높음)
- P/B: 44.40 (업계 평균 대비 높음)
- ROE: 141.71% (매우 높은 수준)

Apple의 밸류에이션은 업계 평균 대비 다소 높은 편이나, 높은 ROE와 브랜드 가치를 고려할 때 정당화될 수 있습니다.

## 7. 투자 의견 및 목표주가
투자의견: 매수
목표주가: 220 USD

Apple의 강력한 브랜드 충성도, 지속적인 혁신, 서비스 부문 성장, 그리고 신규 제품 라인 개발 등을 고려할 때, 장기적인 성장 잠재력이 높다고 판단됩니다. 현재 주가 대비 약 15% 상승한 220 USD를 목표주가로 제시합니다.

## 8. 투자 위험 요인
- 높은 부채 비율 (84.40%)
- 순이익 성장률 감소 (-3.36%)
- 경쟁 심화 및 시장 포화
- 공급망 리스크
- 규제 및 정치적 위험

## 9. 재무제표 요약
### 손익계산서 (단위: 백만 USD)
|항목|2024년|2023년|
|---|---|---|
|총 수익|391,035|383,285|
|영업이익|123,216|114,301|
|순이익|93,736|96,995|
|EPS(희석)|6.08|6.13|

### 재무상태표 및 현금흐름표
상세 정보 없음

## 결론
Apple Inc.는 강력한 브랜드 가치, 지속적인 혁신 능력, 그리고 견고한 재무 성과를 바탕으로 장기 투자자들에게 매력적인 투자 대상입니다. 다만, 순이익 감소 추세와 높은 부채 비율 등의 리스크 요인을 고려해야 합니다. 보수적인 투자자들은 이러한 위험 요인들을 주의 깊게 모니터링하면서 단계적인 매수 전략을 고려할 수 있습니다. 성장 지향적인 투자자들에게는 Apple의 신규 제품 라인과 서비스 부문 성장에 따른 상승 잠재력이 매력적일 것입니다. 종합적으로, Apple 주식은 장기적인 관점에서 포트폴리오에 포함시킬 만한 가치가 있는 투자 대상으로 판단됩니다.